In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import math 

In [2]:
#Updates/Tasks to make Code Neater

#1. Make code calculate annualized GMV given a general N-month solution,
#rather than the hard-coded 3 months we have. 

#2. Find smart/efficient way to calculate product concentration
#i.e. %sales of a brand from top 3 products

# Methodology

There are 3 main parts to our code below. 
    - First, we create a filtered subset of the products_daily data and filter out brands from there. 
    - Second, from this intially filtered list of brands, we look at the ENTIRE products_daily and check for other stuff, filtering out more brands still. 
    - Lastly, we have our final list of brands and want to give more color to the brand's performance by calculating some extra info. 

# Inputs

In [3]:
#Input the values you would like to filter by:

# Number of months we have data for
num_months = 4
scaling_n = 12 / num_months #used to extrapolate to annual figures

#Filter 1: Categories
(cat_1_on, cat_2_on, cat_3_on) = True, False, False
cat_lst_1 = ["Accessories", "Bag", "Men's fashion", 
               "Men's Shoes", "Women's fashion", 
               "Women's Shoes"]
cat_lst_2 = []
cat_lst_3 = []

#Filter 2: Times (Update-status, how established product is)
string = "1/1/2021"
element = datetime.strptime(string,"%d/%m/%Y") 
unix_time_now = datetime.timestamp(element) 

#unix_time_now = 1609682748 #Change based on most recent month

established_on = False
min_years = 1

#Filter 3: Annual Brand GMV
min_gmv_on = True
min_gmv = 70000

#Filter 4: Brand Avg Price 
min_unit_price_on = True
min_unit_price = 300

#Filter 5: GMV Concentration
# Omitted b/c Fashion has so many SKU's

#Filter 6: Avg Product Rating
min_avg_star_on = True
min_avg_star = 4.0

#Filter 7: Bad Customer Ratings 
max_bad_rating_on = True
max_bad_rating = 0.5

#Filter 8: Brand TOTAL GMV 
min_total_brand_gmv_on = True
min_total_brand_gmv = 2000000 

In [4]:
#Make csv of the applied filters so ppl can understand more easily 

filter_df = pd.DataFrame()
# filter_df["cat_1"] = cat_lst_1
# filter_df["cat_2"] = cat_lst_2
# filter_df["cat_3"] = cat_lst_3
filter_df.loc[0, "Filtered_Annual_Brand_GMV"] = min_gmv
filter_df.loc[0, "Filtered_Avg_Price"] = min_unit_price
filter_df.loc[0, "Filtered_Product_Rating"] = min_avg_star
filter_df.loc[0, "Filtered_Max_Bad_Customer_Rating"] = max_bad_rating
filter_df.loc[0, "Min_Total_Brand_GMV"] = min_total_brand_gmv

In [5]:
filter_df.to_csv("filter_df.csv")

# Import Data

In [6]:
products_daily = pd.read_csv("products_daily_1610201363.csv")

In [7]:
#Change data types in dataframes to conserve memory:
def convert_to_cats(df, to_convert):
    '''
    Converts cols in to_convert_list to categories. 
    
    Inputs:
        df: a DataFrame
        to_convert: a list of column names (as strings)
                    we'd like to convert
    Returns:
        Nothing; modifies dataframe in place
    '''
    for col in df.columns:
        if col in to_convert:
            df[col] = df[col].astype("category")
            
to_convert_products_daily = ["added_at", 'category_one', 
                             'category_two', 'category_three', 
                             'category_one_en', 'category_two_en',
                             'category_three_en', 'category_one_th',
                             'category_two_th', 'category_three_th']

convert_to_cats(products_daily, to_convert_products_daily)

def downcast_numbers(df):
    '''
    Downcasts floats and ints.
    
    Inputs:
        df: a DataFrame objecet. 
    Returns:
        Nothing; modifies list in place
    '''
    for col in df.columns:
        if df[col].dtype == "float":
            df[col] = pd.to_numeric(df[col], downcast="float")
        if df[col].dtype == "int":
            df[col] = pd.to_numeric(df[col], downcast="unsigned")
            
downcast_numbers(products_daily)

In [8]:
#Take out nonselling products we don't care about 
mask = products_daily["gmv"] > 1
products_daily = products_daily[mask]

In [9]:
#Make New Columns to account that our data is a 3-mo. snapshot
products_daily["weighted_star"] = products_daily["rating_star"] * products_daily["sold"]
products_daily["n_month_weighted_star"] = products_daily["weighted_star"] * scaling_n
products_daily["n_month_gmv"] = products_daily["gmv"] * scaling_n
products_daily["n_month_sold"] = products_daily["sold"] * scaling_n
products_daily["n_month_view_count"] = products_daily["view_count"] * scaling_n

#Make untouched copy of products data for later
untouched_products_daily = products_daily.copy(deep=True)

#### Logic for why we have columns like "n_month_gmv": 

We want to calculate annual gmv. If a product is present for all Oct, Nov, Dec, i.e. it has 3 rows in the products_daily, then we'd take the average product_gmv for each row and x12. This is equivalent to taking the row-specific product_gmv, multiplying by 4, then adding up these numbers. 

The general case takes n. When we have last 12 months of data consistently can get rid of this variable

Our method has assumption: if a product sells for only 1 of 3 months, then it will sell for 4 of 12 months.


# Product Filter 1: Category Filter

In [10]:
#Filter out products in irrelevant categories. 
def filter_by_category(df, category_n, ok_cat_lst):
    '''
    Keeps rows in the df that have cats in ok_cat_lst. 
    
    Inputs:
        df: a DataFrame
        category_n: (str) eg. "category_one"
        ok_cat_lst: (lst) of acceptable cat for that 
          cat as numbers eg [50, 26]
    Returns:
        The modified df 
    '''
    
    mask = df[category_n].isin(ok_cat_lst)
    return df[mask]

In [11]:
if cat_1_on:
    products_daily = filter_by_category(products_daily, "category_one_en", cat_lst_1) 

In [12]:
if cat_2_on:
    products_daily = filter_by_category(products_daily, "category_one_en", cat_lst_1) 

In [13]:
if cat_3_on:
    products_daily = filter_by_category(products_daily, "category_one_en", cat_lst_1) 

# Product Filter 2: Keep only Established Products and Products that Sellers are Updating

In [14]:
#Filter out products that have been sold for less than a specified time. 
if established_on:
    min_s = min_years * 365 * 24 * 60 * 60 
    mask = unix_time_now - products_daily["ctime"] > min_s
    products_daily = products_daily[mask]

# Brand Definition

In [15]:
#Get rid of products w/o brand info
mask = products_daily["brand"] != "No Brand(ไม่มียี่ห้อ)"
products_daily = products_daily[mask]

#Make a DataFrame of brands from which we'll start narrowing down. 
brands_list = products_daily["brand"].unique()
brands_list = np.delete(brands_list, np.where(brands_list == ('No Brand(ไม่มียี่ห้อ)')))

In [16]:
#Clean up brands_df
brands_df = pd.DataFrame(brands_list, columns = ["Brand Name"])

brands_df.dropna(inplace=True)

s1 = brands_df["Brand Name"] != "None"
brands_df = brands_df[s1]

s2 = brands_df["Brand Name"] != "0"
brands_df = brands_df[s2] 

In [17]:
#Set up columns for our brands_df

brands_df["Filtered_Brand_Annual_GMV"] = 0 
brands_df["Filtered_Average_Unit_Price"] = 0
brands_df["Filtered_Weighted_Star_Rating"] = 0
brands_df["Filtered_Bad_Rating_Percent"] = 0
brands_df.set_index("Brand Name", inplace = True)

In [18]:
for brand in brands_df.index:
    
    #make a sub-df containing only rows with the correct brand
    my_brand_df = products_daily[products_daily["brand"] == brand]
    my_brand_df = my_brand_df.sort_values(by=["gmv"], ascending=False) 
    
    #Approximated ANNUAL gmv calculation
    brand_gmv = my_brand_df["n_month_gmv"].sum()
    brands_df.loc[brand, "Filtered_Brand_Annual_GMV"] = brand_gmv
    
    #ANNUAL volume calculation
    brand_volume = my_brand_df["n_month_sold"].sum()
    brands_df.loc[brand, "Filtered_Average_Unit_Price"] = brand_gmv / brand_volume

    #ANNUAL Weighted Star Rating
    total_star = my_brand_df["n_month_weighted_star"].sum()
    brands_df.loc[brand, "Filtered_Weighted_Star_Rating"] = total_star / brand_volume
    
    #ANNUAL Bad Rating Count
    bad_rating_count = my_brand_df["rating_count_one"].sum() + my_brand_df["rating_count_two"].sum()
    bad_rating_count *= scaling_n
    total_rating_count = my_brand_df["rating_count_total"].sum() * scaling_n
    
    if total_rating_count != 0:
        val = bad_rating_count / total_rating_count 
    else: 
        val = -1
    
    brands_df.loc[brand, "Filtered_Bad_Rating_Percent"] = val

brands_df["Filtered_Bad_Rating_Percent"] = brands_df["Filtered_Bad_Rating_Percent"].round(decimals=3)

# Brand Filter 1: Brand Filtered GMV

In [19]:
if min_gmv_on:
    brands_df = brands_df[brands_df["Filtered_Brand_Annual_GMV"] >= min_gmv]

# Brand Filter 2: Brand (Filtered) Average Price

In [20]:
if min_unit_price_on:
    brands_df = brands_df[brands_df["Filtered_Average_Unit_Price"] >= min_unit_price]

# Brand Filter 3: GMV Concentration of Top (3) Products

In [21]:
pass
#Skip since Fashion has so many SKU's
# if min_gmv_conc_on:
#     brands_df = brands_df[brands_df["Filtered_GMV_Concentration"] >= min_concentration]

# Brand Filter 4: Average (Filtered) Product Rating

In [22]:
if min_avg_star_on:
    brands_df = brands_df[brands_df["Filtered_Weighted_Star_Rating"] >= min_avg_star]

# Brand Filter 5: (Filtered) Bad Customer Ratings

In [23]:
if max_bad_rating_on:
    brands_df = brands_df[brands_df["Filtered_Bad_Rating_Percent"] <= max_bad_rating]

# Brand Filter 6: Brand GMV (Across All Categories) 

In [24]:
#Using the untouched products data, calculate info pertainting to 
#unfiltered products including: brand gmv across all cats, its 
#predominant cat_2 focus


if min_total_brand_gmv_on: #Always kept on        
    for brand in brands_df.index:

        #make a sub-df containing only rows with the correct brand
        mask_brand = untouched_products_daily["brand"] == brand
        my_brand_df = untouched_products_daily[mask_brand]

        #TOTAL ANNUAL Brand gmv calculation
        total_brand_gmv = my_brand_df["n_month_gmv"].sum() 
        brands_df.loc[brand, "Brand_GMV_All_Cats"] = total_brand_gmv
        
        #Total brand view count 
        total_brand_views = my_brand_df["n_month_view_count"].sum()
        conv_rate = total_brand_gmv / total_brand_views
        brands_df.loc[brand, "Conversion_Rate_All_Cats_(gmv/views)"] = conv_rate
        
        #Finding brand's predominant cat_2
        winning_cat_2 = None
        winning_cat_2_gmv = 0.0

        for cat_2 in my_brand_df["category_two_en"].unique():
            mask_cat_2 = my_brand_df["category_two_en"] == cat_2
            cat_2_df = my_brand_df[mask_cat_2]

            cat_2_gmv = cat_2_df["n_month_gmv"].sum()
            if cat_2_gmv > winning_cat_2_gmv:
                winning_cat_2 = cat_2
                winning_cat_2_gmv = cat_2_gmv
                    
        percent = (winning_cat_2_gmv / total_brand_gmv).round(decimals=3)
        brands_df.loc[brand, "Cat_2_Percent_of_Total_GMV"] = percent 
        brands_df.loc[brand, "Brand_Category_2"] = winning_cat_2

        #print("for brand: ", brand, "Winning cat 2 is: ", winning_cat_2)
        #print("with a cat_2_gmv value of: ", winning_cat_2_gmv)    

In [25]:
#Apply the total brand gmv filter
brands_df = brands_df[brands_df["Brand_GMV_All_Cats"] >= min_total_brand_gmv]

# Filter for Brands that Aren't Fashion

In [26]:
#I.e. avoid the Xioami issue. Check to see if the predominant Cat 2 of the 
#brands are in the correct Cat 1's. Filter out brands that aren't. 

#Pull out relevant Cat 2's from our filtered product data set
good_cat_2 = products_daily["category_two_en"].unique()

#Filter
mask = brands_df["Brand_Category_2"].isin(good_cat_2)
brands_df = brands_df[mask]

# Calculate Additional Info on Recency of Sales for Shortlisted Brands

In [27]:
#From the final list of brands we have, add info on recency ratio 
#eg. percentage of total gmv coming from products established at 
#least 3, 6, 12, 24 months. 

In [28]:
#Calculate times
three_mo = 3 * 30 * 24 * 60 * 60
six_mo = 6 * 30 * 24 * 60 * 60
twelve_mo = 12 * 30 * 24 * 60 * 60
twenty_four_mo = 24 * 30 * 24 * 60 * 60

In [29]:
#Make four dataframes representing slices of the untouched products
#that have sold for more than 3, 6, 12, 24 months. 

mask_3 = unix_time_now - untouched_products_daily["ctime"] > three_mo
timed_df_3 = untouched_products_daily[mask_3]

mask_6 = unix_time_now - untouched_products_daily["ctime"] > six_mo
timed_df_6 = untouched_products_daily[mask_6]

mask_12 = unix_time_now - untouched_products_daily["ctime"] > twelve_mo
timed_df_12 = untouched_products_daily[mask_12]

mask_24 = unix_time_now - untouched_products_daily["ctime"] > twenty_four_mo
timed_df_24 = untouched_products_daily[mask_24]

In [30]:
#Calculate GMV generated from products that are N months and older 

for brand in brands_df.index:
    #make a sub-df containing only rows with the correct brand
    my_brand_df = timed_df_3[timed_df_3["brand"] == brand]
    
    #gmv calculation
    brand_gmv_3 = my_brand_df["n_month_gmv"].sum()
    brands_df.loc[brand, "%Total GMV from products 3 months+"] = brand_gmv_3

for brand in brands_df.index:
    #make a sub-df containing only rows with the correct brand
    my_brand_df = timed_df_6[timed_df_6["brand"] == brand]
    
    #gmv calculation
    brand_gmv_6 = my_brand_df["n_month_gmv"].sum()
    brands_df.loc[brand, "%Total GMV from products 6 months+"] = brand_gmv_6
    
for brand in brands_df.index:
    #make a sub-df containing only rows with the correct brand
    my_brand_df = timed_df_12[timed_df_12["brand"] == brand]
    
    #gmv calculation
    brand_gmv_12 = my_brand_df["n_month_gmv"].sum()
    brands_df.loc[brand, "%Total GMV from products 12 months+"] = brand_gmv_12
    
for brand in brands_df.index:
    #make a sub-df containing only rows with the correct brand
    my_brand_df = timed_df_24[timed_df_24["brand"] == brand]
    
    #gmv calculation
    brand_gmv_24 = my_brand_df["n_month_gmv"].sum()
    brands_df.loc[brand, "%Total GMV from products 24 months+"] = brand_gmv_24


In [31]:
#For each brand, we have now calculated GMV from products that
#are at least 3, 6, 12, 24 months old. Now, change it to % of total. 


for brand in brands_df.index:
    percentage = brands_df.loc[brand, "%Total GMV from products 3 months+"] / brands_df.loc[brand, 
                                                                                   "Brand_GMV_All_Cats"]
    brands_df.loc[brand, "%Total GMV from products 3 months+"] = round(percentage, 3)
    
for brand in brands_df.index:
    percentage = brands_df.loc[brand, "%Total GMV from products 6 months+"] / brands_df.loc[brand, 
                                                                                   "Brand_GMV_All_Cats"]
    brands_df.loc[brand, "%Total GMV from products 6 months+"] = round(percentage, 3)
    
for brand in brands_df.index:
    percentage = brands_df.loc[brand, "%Total GMV from products 12 months+"] / brands_df.loc[brand, 
                                                                                   "Brand_GMV_All_Cats"]
    brands_df.loc[brand, "%Total GMV from products 12 months+"] = round(percentage, 3)
    
for brand in brands_df.index:
    percentage = brands_df.loc[brand, "%Total GMV from products 24 months+"] / brands_df.loc[brand, 
                                                                                   "Brand_GMV_All_Cats"]
    brands_df.loc[brand, "%Total GMV from products 24 months+"] = round(percentage, 3)

# Calculate Additional Info on Discounts for Shortlisted Brands

In [32]:
#From the final list of brands we have, add info on how 
#much they discount on ALL their products. 

In [33]:
#First, fill in 0 values in "price_before_discount" with "price" value
#This means there was no discount
conditions = [untouched_products_daily['price_before_discount'].eq(0)]
choices = [untouched_products_daily["price"]]
untouched_products_daily['price_before_discount'] = np.select(conditions,choices,default=untouched_products_daily['price_before_discount'])

In [34]:
#Now, create a n-month-full-priced-gmv 
untouched_products_daily["full_priced_gmv"] = untouched_products_daily["price_before_discount"] * untouched_products_daily["sold"]
untouched_products_daily["n_mo_full_priced_gmv"] = untouched_products_daily["full_priced_gmv"] * scaling_n

In [35]:
#Now, for each brand calculate the would-be full_priced annual gmv. (go for each row)

for brand in brands_df.index:
    
    #make a sub-df containing only rows with the correct brand
    my_brand_df = untouched_products_daily[untouched_products_daily["brand"] == brand]
    
    #Approximated ANNUAL gmv calculation
    brand_full_priced_gmv = my_brand_df["n_mo_full_priced_gmv"].sum()
    brands_df.loc[brand, "Brand_Full_Priced_GMV"] = brand_full_priced_gmv #this number is inherently an annual

In [36]:
#Divide the Brand_GMV_All_Cats (how much brand actually sold) by 
#Brand_Full_Priced_GMV (what it would've sold in an ideal, non-discount world). 

brands_df["Average_Disc"] = 1 - brands_df["Brand_GMV_All_Cats"] / brands_df["Brand_Full_Priced_GMV"]

# Output: Brands that Meet Our Criterion

In [37]:
brands_df = brands_df.rename(columns={"Cat_2_Percent_of_Total_GMV" : "Cat_2_Contr_to_Total_GMV"})
brands_df.sort_index(inplace=True)

In [38]:
#Reorganize the outputted df
brands_df = brands_df[['Filtered_Brand_Annual_GMV', 'Brand_Full_Priced_GMV', 
                       'Filtered_Average_Unit_Price', 
                       'Filtered_Weighted_Star_Rating',
                       'Filtered_Bad_Rating_Percent', 'Brand_GMV_All_Cats',
                       'Conversion_Rate_All_Cats_(gmv/views)', 'Cat_2_Contr_to_Total_GMV',
                       'Brand_Category_2', '%Total GMV from products 3 months+',
                       '%Total GMV from products 6 months+',
                       '%Total GMV from products 12 months+',
                       '%Total GMV from products 24 months+',
                       'Average_Disc']]

In [39]:
display(brands_df)
#NOTE: Anything before the Brand_GMV_All_Cats uses FILTERED product data 
#i.e. data where we've the products to count for by category, and time. 

,Filtered_Brand_Annual_GMV,Brand_Full_Priced_GMV,Filtered_Average_Unit_Price,Filtered_Weighted_Star_Rating,Filtered_Bad_Rating_Percent,Brand_GMV_All_Cats,Conversion_Rate_All_Cats_(gmv/views),Cat_2_Contr_to_Total_GMV,Brand_Category_2,%Total GMV from products 3 months+,%Total GMV from products 6 months+,%Total GMV from products 12 months+,%Total GMV from products 24 months+,Average_Disc
Brand Name,,,,,,,,,,,,,,
ARROW (แอร์โร่ว์),5114541.0,9568380.0,1261.914878,4.819486,0.010,5137005.0,26.391119,0.956,Shorts,1.000,1.000,0.010,0.010,0.463127
ARROW(แอร์โร่ว์),2536911.0,4907805.0,574.872196,4.756042,0.014,2564601.0,9.929230,0.314,Shirt,0.999,0.714,0.182,0.177,0.477444
Adidas(อดิดาส),130294194.0,281039298.0,695.091433,4.687243,0.014,162544398.0,5.470588,0.418,Lace-up sneakers,0.791,0.530,0.244,0.060,0.421631
Aerosoft(เอโรซอฟท์),4395297.0,7381389.0,469.884221,4.748939,0.005,4395297.0,13.412359,0.459,Casual shoes,0.985,0.696,0.185,0.081,0.404543
Agold(อะโกลด์),17350425.0,17390055.0,2829.488748,4.812923,0.009,17350425.0,40.765443,1.000,Gold,0.930,0.923,0.916,0.851,0.002279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ห้างขายทองสุวรรณภูมิ3,8117088.0,10559829.0,1744.484848,4.725249,0.022,8117088.0,6.993812,1.000,Gold,1.000,1.000,0.647,0.000,0.231324
ห้างทองรวมสินไทยขอนแก่น,61955883.0,80197344.0,3631.433269,4.931094,0.001,61955883.0,49.348992,0.992,Gold,0.995,0.995,0.973,0.000,0.227457
ห้างทองแม่ทองสุก,7369200.0,9094350.0,8558.885017,4.842377,0.000,7369200.0,45.244235,1.000,Gold,0.887,0.853,0.000,0.000,0.189695


In [40]:
#Make numbers cleaner
brands_df["Brand_Full_Priced_GMV"] = brands_df["Brand_Full_Priced_GMV"].astype("int")
brands_df["Filtered_Average_Unit_Price"] = brands_df["Filtered_Average_Unit_Price"].astype("int")
brands_df["Conversion_Rate_All_Cats_(gmv/views)"] = brands_df["Conversion_Rate_All_Cats_(gmv/views)"].round(decimals=1)
brands_df["Filtered_Weighted_Star_Rating"] = brands_df["Filtered_Weighted_Star_Rating"].round(decimals=2)
brands_df["Filtered_Bad_Rating_Percent"] = brands_df["Filtered_Bad_Rating_Percent"].round(decimals=2)
brands_df["Cat_2_Contr_to_Total_GMV"] = brands_df["Cat_2_Contr_to_Total_GMV"].round(decimals=2)
brands_df["%Total GMV from products 3 months+"] = brands_df["%Total GMV from products 3 months+"].round(decimals=2)
brands_df["%Total GMV from products 6 months+"] = brands_df["%Total GMV from products 6 months+"].round(decimals=2)
brands_df["%Total GMV from products 12 months+"] = brands_df["%Total GMV from products 12 months+"].round(decimals=2)
brands_df["%Total GMV from products 24 months+"] = brands_df["%Total GMV from products 24 months+"].round(decimals=2)
brands_df["Average_Disc"] = brands_df["Average_Disc"].round(decimals=2)
brands_df["Brand_GMV_All_Cats"] = brands_df["Brand_GMV_All_Cats"].astype("int")

In [41]:
#Convert to csv if desired. 
brands_df.to_csv("fashion_brands_shortlist.csv")

#to make presentable, convert some columns to percentages in Excel. 